###  神经网络

Autograd实现了反向传播功能，但是直接用来写深度学习的代码在很多情况下还是稍显复杂，torch.nn是专门为神经网络设计的模块化接口。nn构建于 Autograd之上，可用来定义和运行神经网络。nn.Module是nn中最重要的类，可把它看成是一个网络的封装，包含网络各层定义以及forward方法，调用forward(input)方法，可返回前向传播的结果。下面就以最早的卷积神经网络：LeNet为例，来看看如何用`nn.Module`实现。LeNet的网络结构如图2-7所示。

![图2-7:LeNet网络结构](imgs/nn_lenet.png)

这是一个基础的前向传播(feed-forward)网络: 接收输入，经过层层传递运算，得到输出。

#### 定义网络

定义网络时，需要继承`nn.Module`，并实现它的forward方法，把网络中具有可学习参数的层放在构造函数`__init__`中。如果某一层(如ReLU)不具有可学习的参数，则既可以放在构造函数中，也可以不放，但建议不放在其中，而在forward中使用`nn.functional`代替。

Conv2d的参数
[ channels, output, height_2, width_2 ]
<br>channels, 通道数，当前层的深度
<br>output 输出的深度      
<br>height_2, 过滤器filter的高
<br>width_2, 过滤器filter的宽                                                       

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        # nn.Module子类的函数必须在构造函数中执行父类的构造函数
        # 下式等价于nn.Module.__init__(self)
        super(Net, self).__init__()
        
        # 卷积层1 输入图片为单通道, 输出通道数为6，卷积核为5*5
        #todo
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        # 卷积层2 表示输入图片为6通道, 输出通道数为16，卷积核为5*5
        #todo
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        # 仿射层/全连接层1，输入维度为：16*5*5，输出维度为：120
        #todo
        self.fc1 = nn.Linear(16*5*5, 120)
        # 仿射层/全连接层2，输入维度为：120，输出维度为：84
        #todo
        self.fc2 = nn.Linear(120, 84)
        # 仿射层/全连接层3，输入维度为：84，输出维度为：10
        #todo
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x): 
        # 卷积 -> 激活 -> 池化 
        # tip1：F.max_pool2d
        # tip2：每一层卷积后都需要激活和池化
        #todo
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        #todo
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        #卷积完成后对输入进行reshape，然后输入到全连接中
        #tip1：每层全连接后都需要激活
        # tip2：reshape，‘-1’表示自适应 
        #todo
        x = x.view(-1, 16*5*5)
        #todo
        x = F.relu(self.fc1(x))
        #todo
        x = F.relu(self.fc2(x))
        #todo
        x = F.relu(self.fc3(x))
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


只要在nn.Module的子类中定义了forward函数，backward函数就会自动被实现(利用`autograd`)。在`forward` 函数中可使用任何tensor支持的函数，还可以使用if、for循环、print、log等Python语法，写法和标准的Python写法一致。

网络的可学习参数通过`net.parameters()`返回，`net.named_parameters`可同时返回可学习的参数及名称。

In [10]:
params = list(net.parameters())
print(len(params))

10


In [11]:
for name,parameters in net.named_parameters():
    print(name,':',parameters.size())

conv1.weight : torch.Size([6, 1, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])


conv1.weight : torch.Size([6, 1, 5, 5])
<br>conv1.bias : torch.Size([6])
<br>conv2.weight : torch.Size([16, 6, 5, 5])
<br>conv2.bias : torch.Size([16])
<br>fc1.weight : torch.Size([120, 400])
<br>fc1.bias : torch.Size([120])
<br>fc2.weight : torch.Size([84, 120])
<br>fc2.bias : torch.Size([84])
<br>fc3.weight : torch.Size([10, 84])
<br>fc3.bias : torch.Size([10])

forward函数的输入和输出都是Tensor。

In [47]:
import torch as t
input = t.randn(1, 1, 32, 32)
out = net(input)
out.size()

torch.Size([1, 10])

In [48]:
# 所有参数的梯度清零
#tip:zero_grad() 
#todo
net.zero_grad()
# 反向传播
#tip:backward()
#todo
out.backward(t.ones_like(out))

需要注意的是，torch.nn只支持mini-batches，不支持一次只输入一个样本，即一次必须是一个batch。但如果只想输入一个样本，则用 `input.unsqueeze(0)`将batch_size设为１。例如 `nn.Conv2d` 输入必须是4维的，形如$nSamples \times nChannels \times Height \times Width$。可将nSample设为1，即$1 \times nChannels \times Height \times Width$。


<br>out.backward(t.ones_like(out))
<br>backward默认不可以使用张量，如果使用张量就需要输入一个大小相同的张量作为参数。

#### 损失函数

nn实现了神经网络中大多数的损失函数，例如nn.MSELoss用来计算均方误差，nn.CrossEntropyLoss用来计算交叉熵损失。

In [49]:
output = net(input)
target = t.arange(0,10).view(1,10).float()

#Loss设计
#tip MSELoss
#todo criterion = 
criterion = nn.MSELoss()
loss = criterion(output, target)
loss # loss是个scalar

tensor(28.3205, grad_fn=<MseLossBackward>)

如果对loss进行反向传播溯源(使用`gradfn`属性)，可看到它的计算图如下：

```
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d  
      -> view -> linear -> relu -> linear -> relu -> linear 
      -> MSELoss
      -> loss
```

当调用`loss.backward()`时，该图会动态生成并自动微分，也即会自动计算图中参数(Parameter)的导数。

In [50]:
# 运行.backward，观察调用之前和调用之后的grad
# 把net中所有可学习参数的梯度清零
#tip zero_grad()
#todo
net.zero_grad()
print('反向传播之前 conv1.bias的梯度')
print(net.conv1.bias.grad)
#反向传播
#tip:loss.backward()
loss.backward()
print('反向传播之后 conv1.bias的梯度')
print(net.conv1.bias.grad)

反向传播之前 conv1.bias的梯度
tensor([0., 0., 0., 0., 0., 0.])
反向传播之后 conv1.bias的梯度
tensor([-0.0193, -0.0510, -0.0725, -0.0509, -0.0630, -0.0162])


#### 优化器

在反向传播计算完所有参数的梯度后，还需要使用优化方法来更新网络的权重和参数，例如随机梯度下降法(SGD)的更新策略如下：
```
weight = weight - learning_rate * gradient
```

手动实现如下：

```python
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)# inplace 减法
```

`torch.optim`中实现了深度学习中绝大多数的优化方法，例如RMSProp、Adam、SGD等，更便于使用，因此大多数时候并不需要手动写上述代码。

In [54]:
import torch.optim as optim
#新建一个优化器，指定要调整的参数和学习率
#tip:SGD优化器
#todo optimizer = 
optimizer = optim.SGD(net.parameters(), lr=0.1,
                      momentum=0.5)
# 在训练过程中
# 先梯度清零(与net.zero_grad()效果一样)
optimizer.zero_grad() 

# 计算损失
output = net(input)
loss = criterion(output, target)

#反向传播
loss.backward()

#更新参数
optimizer.step()

In [56]:
import numpy as np
import torch
import torch.nn.functional as F
def log_softmax(x):
    x_exp = np.exp(x)
    s = x_exp / np.sum(x_exp, axis=1, keepdims=True)
    s = np.log(s)
    return s

a = [[5, 2, -1, 3],[0,-2,-3, -2]]  # (2*4)
a = np.array(a)
a = log_softmax(a)
print(a)
a = [[5, 2, -1, 3],[0,-2,-3, -2]]
a = torch.Tensor(a)
a = F.log_softmax(a, dim=1)
print(a)

[[-0.17193539 -3.17193539 -6.17193539 -2.17193539]
 [-0.27797837 -2.27797837 -3.27797837 -2.27797837]]
tensor([[-0.1719, -3.1719, -6.1719, -2.1719],
        [-0.2780, -2.2780, -3.2780, -2.2780]])
